In [19]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, datasets, linear_model, model_selection
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from cmath import sqrt

dataframe=pd.read_csv('Dataset.csv')

Видалимо всі рядки з порожніми даними, та замінимо стовпчики з категоріальними даними на закодовані еквіваленти.

In [20]:
dataframe.drop(dataframe.index[dataframe.isna().any(axis=1)].tolist(),axis=0,inplace=True)
income_dataframe=dataframe.filter(['target_sub_ltv_day30','target_iap_ltv_day30','target_ad_ltv_day30','target_full_ltv_day30'],axis=1)
dataframe.drop(columns=['target_sub_ltv_day30','target_iap_ltv_day30','target_ad_ltv_day30','target_full_ltv_day30'],axis=1,inplace=True)

categorical_dataframe=dataframe.filter(['platform','media_source','install_date', 'country_code'],axis=1)
dataframe.drop(columns=['media_source','install_date', 'platform','country_code'],axis=1,inplace=True)
dummies_platform=pd.get_dummies(categorical_dataframe['platform'],drop_first=True)
dataframe=pd.concat([dataframe,dummies_platform],axis='columns')
print(dataframe)

         total_sessions_day0  total_sessions_day1  total_sessions_day3  \
0                        1.0                  1.0                  1.0   
1                        1.0                  1.0                  1.0   
2                        1.0                  2.0                  2.0   
3                        1.0                  1.0                  1.0   
4                        1.0                  1.0                  1.0   
...                      ...                  ...                  ...   
1467827                 11.0                 11.0                 11.0   
1467828                  2.0                  3.0                  6.0   
1467829                  3.0                  5.0                 11.0   
1467830                  5.0                  5.0                  5.0   
1467831                  6.0                  7.0                  7.0   

         total_sessions_day7  chapters_finished_day0  chapters_finished_day1  \
0                        1.0   

In [21]:
categorical_dataframe['country_frequency']=categorical_dataframe['country_code'].map(categorical_dataframe['country_code'].value_counts())
categorical_dataframe['media_frequency']=categorical_dataframe['media_source'].map(categorical_dataframe['media_source'].value_counts())

In [22]:
new_dataframe=pd.concat([dataframe,categorical_dataframe['country_frequency']],axis='columns')
new_dataframe=pd.concat([new_dataframe,categorical_dataframe['media_frequency']],axis='columns')
print(new_dataframe['media_frequency'])
print(new_dataframe['country_frequency'])

0          238049
1           63231
2          318053
3          238049
4           23379
            ...  
1467827    318053
1467828    238049
1467829    238049
1467830    728849
1467831     23379
Name: media_frequency, Length: 1466458, dtype: int64
0            2650
1          387032
2          387032
3          387032
4           14680
            ...  
1467827    387032
1467828    387032
1467829    387032
1467830    387032
1467831    387032
Name: country_frequency, Length: 1466458, dtype: int64


Натренуємо модель на всіх фічах для того, щоб на основі абсолютних значень коефіцієнтів впорядкувати фічі за "важливістю".

In [23]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(new_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
model=linear_model.LinearRegression(fit_intercept=1)
model.fit(X_train, y_train)
coef=[]
for i in range(len(model.coef_)):
    coef.append((new_dataframe.columns.values[i], abs(model.coef_[i])))
coef=sorted(coef, key=lambda tup: tup[1])
coef.reverse()
for i in range(len(model.coef_)):
    print(coef[i])

('ad_ltv_day3', 4.232881568559846)
('ad_ltv_day1', 2.529826966696556)
('app_iap_ltv_day3', 1.6242945815182983)
('app_sub_ltv_day3', 1.3287143593622581)
('app_sub_ltv_day0', 1.2339492843843896)
('app_iap_ltv_day1', 0.4020620645543068)
('retained_day7', 0.3440335354169588)
('app_sub_ltv_day1', 0.29470365889038397)
('retained_day3', 0.21436866713094105)
('ad_ltv_day0', 0.17166421492809691)
('chapters_closed_day7', 0.16492565728304556)
('chapters_closed_day3', 0.15528623039122808)
('app_iap_ltv_day0', 0.10636515581753803)
('retained_day1', 0.09151902116798011)
('chapters_finished_day3', 0.08698807341390503)
('tickets_spent_day1', 0.08290697999694416)
('chapters_opened_session9', 0.07121369931317506)
('chapters_finished_session9', 0.06381642511385147)
('chapters_closed_session9', 0.06161245414934646)
('chapters_opened_day1', 0.06074847158671977)
('chapters_finished_day7', 0.05494510364875676)
('tickets_spent_day3', 0.05399823210424198)
('tickets_spent_session1', 0.039978839403404576)
('chap

Будемо поступово(за "важливістю") додавати фічі в модель і у випадку покращення метрики MAE залишати їх в моделі.

In [24]:
features=[]
min=1000000
best_features=[]
for x in coef:
    name=x[1]
    features.append(x[0])
    training_dataframe=new_dataframe.filter(features,axis='columns')
    X_train, X_test, y_train, y_test = model_selection.train_test_split(training_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
    model=linear_model.LinearRegression(fit_intercept=1)    
    model.fit(X_train, y_train)
    predictions=model.predict(X_test)
    a=mean_absolute_error(predictions,y_test)
    print(x[0],a,features)
    if a<min:
        best_features=features.copy()
        min=a
    else:
        features.remove(x[0])

    


ad_ltv_day3 0.3779301825795212 ['ad_ltv_day3']
ad_ltv_day1 0.375992132681524 ['ad_ltv_day3', 'ad_ltv_day1']
app_iap_ltv_day3 0.1314366863943583 ['ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3']
app_sub_ltv_day3 0.12536549441354364 ['ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3', 'app_sub_ltv_day3']
app_sub_ltv_day0 0.12490378667437273 ['ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3', 'app_sub_ltv_day3', 'app_sub_ltv_day0']
app_iap_ltv_day1 0.1168624863942858 ['ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3', 'app_sub_ltv_day3', 'app_sub_ltv_day0', 'app_iap_ltv_day1']
retained_day7 0.14061485189942097 ['ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3', 'app_sub_ltv_day3', 'app_sub_ltv_day0', 'app_iap_ltv_day1', 'retained_day7']
app_sub_ltv_day1 0.1189459011164853 ['ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3', 'app_sub_ltv_day3', 'app_sub_ltv_day0', 'app_iap_ltv_day1', 'app_sub_ltv_day1']
retained_day3 0.11865930429439658 ['ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3', 'app_sub

In [25]:
print(best_features,min)

['ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3', 'app_sub_ltv_day3', 'app_sub_ltv_day0', 'app_iap_ltv_day1', 'ad_ltv_day0', 'app_iap_ltv_day0', 'chapters_closed_day1', 'chapters_finished_session3'] 0.11279104489753768


In [26]:
training_dataframe=new_dataframe.filter(best_features,axis='columns')
X_train, X_test, y_train, y_test = model_selection.train_test_split(training_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
model=linear_model.LinearRegression(fit_intercept=1)    
model.fit(X_train, y_train)
predictions=model.predict(X_test)
print(mean_absolute_error(predictions,y_test))
print(sqrt(mean_squared_error(predictions,y_test)))

0.11805826573164349
(1.279283422416669+0j)


Водночас бачимо, що модель з фічами 'ad_ltv_day3', 'ad_ltv_day1', 'app_iap_ltv_day3' має майже таку ж метрику MAE. Враховуючи той факт, що фічі 'ad_ltv_day3' та 'ad_ltv_day1' корелюють (як видно з EDA) то можна спробувати викинути одну з них. Викинемо 'ad_ltv_day1', адже вона має меншу "важливість".

In [30]:
training_dataframe=new_dataframe.filter(['ad_ltv_day3','app_iap_ltv_day3'],axis='columns')
X_train, X_test, y_train, y_test = model_selection.train_test_split(training_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)
model=linear_model.LinearRegression(fit_intercept=1)    
model.fit(X_train, y_train)
predictions=model.predict(X_test)
print(mean_absolute_error(predictions,y_test))
print(sqrt(mean_squared_error(predictions,y_test)))

0.14219378741383495
(1.2291876020432113+0j)


Бачимо що метрика MAE не набагато більша за модель з best_features тому виберемо простішу модель.

In [17]:
print(model.coef_)
print(model.intercept_)

[3.30642215 1.47660667]
-0.0074066380856026015
